In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os

In [2]:
db_path = "/kaggle/input/olist-database/olist.db"

conn = sqlite3.connect(db_path)

In [3]:
#  Commandes récentes avec retard
query_lateorders = """
WITH last_order_date AS (
    SELECT MAX(order_purchase_timestamp) AS max_order_date 
    FROM orders
),
recent_orders AS (
    SELECT 
        o.order_id,
        c.customer_unique_id,
        o.order_delivered_customer_date,
        o.order_estimated_delivery_date,
        (JULIANDAY(o.order_delivered_customer_date) - JULIANDAY(o.order_estimated_delivery_date)) AS delay_days
    FROM 
        orders o
    JOIN 
        customers c ON o.customer_id = c.customer_id
    CROSS JOIN 
        last_order_date
    WHERE 
        o.order_status NOT IN ('canceled', 'unavailable')
        AND (JULIANDAY(last_order_date.max_order_date) - JULIANDAY(o.order_purchase_timestamp)) <= 90
        AND JULIANDAY(o.order_delivered_customer_date) > JULIANDAY(o.order_estimated_delivery_date)
)
SELECT 
    order_id, 
    customer_unique_id, 
    delay_days
FROM 
    recent_orders
WHERE 
    delay_days >= 3;
"""

# Vendeurs générant plus de 100 000 Real en chiffre d’affaires
query_sellers = """
WITH seller_revenue AS (
    SELECT 
        s.seller_id,
        SUM(oi.price) AS total_revenue
    FROM 
        order_items oi
    JOIN 
        sellers s ON oi.seller_id = s.seller_id
    JOIN 
        orders o ON oi.order_id = o.order_id
    WHERE 
        o.order_status = 'delivered'
    GROUP BY 
        s.seller_id
)
SELECT 
    seller_id,
    total_revenue
FROM 
    seller_revenue
WHERE 
    total_revenue > 100000;
"""

#  Nouveaux vendeurs très engagés
query_newsellers = """
WITH last_order_date AS (
    SELECT MAX(order_purchase_timestamp) AS max_order_date 
    FROM orders
),
first_sale_date AS (
    SELECT 
        oi.seller_id,
        MIN(o.order_purchase_timestamp) AS first_sale_date
    FROM 
        order_items oi
    JOIN 
        orders o ON oi.order_id = o.order_id
    GROUP BY 
        oi.seller_id
),
recent_sellers AS (
    SELECT 
        fs.seller_id,
        fs.first_sale_date,
        (JULIANDAY(l.max_order_date) - JULIANDAY(fs.first_sale_date)) AS seller_age_days
    FROM 
        first_sale_date fs
    CROSS JOIN 
        last_order_date l
    WHERE 
        (JULIANDAY(l.max_order_date) - JULIANDAY(fs.first_sale_date)) <= 90
),
seller_activity AS (
    SELECT 
        rs.seller_id,
        rs.first_sale_date,
        COUNT(oi.order_item_id) AS total_products_sold
    FROM 
        recent_sellers rs
    JOIN 
        order_items oi ON rs.seller_id = oi.seller_id
    GROUP BY 
        rs.seller_id, rs.first_sale_date
)
SELECT 
    seller_id,
    total_products_sold,
    first_sale_date
FROM 
    seller_activity
WHERE 
    total_products_sold > 30;
"""

# Les 5 codes postaux avec le pire review score moyen
query_worstreviewscore = """
WITH last_order_date AS (
    SELECT MAX(order_purchase_timestamp) AS max_order_date 
    FROM orders
),
recent_reviews AS (
    SELECT 
        c.customer_zip_code_prefix,
        r.review_score
    FROM 
        orders o
    JOIN 
        customers c ON o.customer_id = c.customer_id
    JOIN 
        order_reviews r ON o.order_id = r.order_id
    CROSS JOIN 
        last_order_date
    WHERE 
        (JULIANDAY(last_order_date.max_order_date) - JULIANDAY(o.order_purchase_timestamp)) <= 365
),
zipcode_reviews AS (
    SELECT 
        customer_zip_code_prefix,
        COUNT(review_score) AS review_count,
        AVG(review_score) AS avg_review_score
    FROM 
        recent_reviews
    GROUP BY 
        customer_zip_code_prefix
    HAVING 
        review_count > 30
)
SELECT 
    customer_zip_code_prefix,
    review_count,
    avg_review_score
FROM 
    zipcode_reviews
ORDER BY 
    avg_review_score ASC
LIMIT 5;
"""

In [4]:
result_lateorders = pd.read_sql_query(query_lateorders, conn)
display(result_lateorders)

,order_id,customer_unique_id,delay_days
0,cfa4fa27b417971e86d8127cb688712f,cadb385cc1299451df18febff40aabc9,7.070613
1,234c056c50619f48da64f731c48242b4,2b18cd5c428106cc024ee2cc951a0b68,9.760208
2,7f579e203c931f3e8410103359c6d523,bddbce40ba8847305fb9858313934193,4.841516
3,cb6e441ff2ef574ce08d3709426f88ec,ad9ea4d00c39f9f3213f2cb69b1bf3a8,3.049977
4,03720fdc92032ee4abd471d172006ab0,e6fdeb2ae78e71035c9db4c86e778455,4.008241
...,...,...,...
302,f6ac44bbece0129b596dfc21aede3c20,4a7da70ea95397be694201c9bf00ce5a,7.755359
303,874a7690bc049bd4ce210d195bdfff7b,4d3cd98800102cadf3a108f741bdab4a,5.660278
304,587e32dd528769d669d539531d32aeb5,e2538fcd738167049ed245b8f2576bcb,6.397581
305,3adb141ba4bd69dd7fe8d3fb733c6b74,7c5750a9ae14793d13ed61b94694963b,4.751991


In [5]:
result_sellers = pd.read_sql_query(query_sellers, conn)
display(result_sellers)

,seller_id,total_revenue
0,1025f0e2d44d7041d6cf58b6550e0bfa,138208.56
1,1f50f920176fa81dab994f9023523100,106655.71
2,46dc3b2cc0980fb8ec44634e21d2718e,122811.38
3,4869f7a5dfa277a7dca6462dcf3b52b2,226987.93
4,4a3ca9315b744ce9f8e9374361493884,196882.12
5,53243585a1d6dc2643021fd1853d8905,217940.44
6,5dceca129747e92ff8ef7a997dc4f8ca,111126.73
7,620c87c171fb2a6dd6e8bb4dec959fc6,112461.50
8,6560211a19b47992c3666cc44a7e94c0,120702.83
9,7a67c85e85bb2ce8582c35f2203ad736,139658.69


In [6]:
result_newsellers = pd.read_sql_query(query_newsellers, conn)
display(result_newsellers)

,seller_id,total_products_sold,first_sale_date
0,81f89e42267213cb94da7ddc301651da,52,2018-08-08 12:45:12
1,d13e50eaa47b4cbe9eb81465865d8cfc,69,2018-08-04 09:09:37


In [7]:
result_worstreviewscore = pd.read_sql_query(query_worstreviewscore, conn)
display(result_worstreviewscore)

,customer_zip_code_prefix,review_count,avg_review_score
0,22753,47,2.808511
1,22770,37,3.135135
2,22793,90,3.233333
3,21321,36,3.277778
4,22780,37,3.351351
